##### 문자 생성 

- [1] 모듈로딩 및 데이터 준비<hr>

In [1]:
### ===> 모듈로딩
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

In [2]:
### ===> 데이터 준비
text = "To be, or not to be, that is the question. Whether 'tis nobler in the mind to suffer. " \
    "The slings and arrows of outrageous fortune, or to take arms against a sea of troubles."


In [30]:
list("ABC")

['A', 'B', 'C']

- [2] 데이터 전처리 <hr>

In [31]:
### ===> 문자 집합 생성
chars = sorted(list(set(text)))
print(f'chars => {chars}')

vocab_size = len(chars)

char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for i, ch in enumerate(chars)}


chars => [' ', "'", ',', '.', 'T', 'W', 'a', 'b', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'q', 'r', 's', 't', 'u', 'w']


In [32]:
print(len(char_to_idx), char_to_idx, sep='\n')

25
{' ': 0, "'": 1, ',': 2, '.': 3, 'T': 4, 'W': 5, 'a': 6, 'b': 7, 'd': 8, 'e': 9, 'f': 10, 'g': 11, 'h': 12, 'i': 13, 'k': 14, 'l': 15, 'm': 16, 'n': 17, 'o': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'w': 24}


In [27]:
### ===> 데이터 전처리 함수
def prepare_data(text, sequence_length):
    input_seq, target_seq = [], []
    
    # 현재 문자와 다음 문자를 입력-타겟으로 저장
    for i in range(len(text) - sequence_length):
        input_seq.append([char_to_idx[ch] for ch in text[i:i + sequence_length]])
        print(f'input_seq[{i}] : {text[i:i + sequence_length]} => {input_seq[i]}')
        
        target_seq.append(char_to_idx[text[i + sequence_length]])
        print(f'target_seq[{i}]: {text[i + sequence_length]} => {target_seq[i]}')
        
    return np.array(input_seq), np.array(target_seq)

In [34]:
### ===> 테스트 
X, y = prepare_data(text, sequence_length=3)
text = "To be, or not to be, that is the question. Whether 'this nobler in the mind to suffer. " \
    "The slings and arrows of outrageous fortune, or to take arms against a sea of troubles."


input_seq[0] : To  => [4, 18, 0]
target_seq[0]: b => 7
input_seq[1] : o b => [18, 0, 7]
target_seq[1]: e => 9
input_seq[2] :  be => [0, 7, 9]
target_seq[2]: , => 2
input_seq[3] : be, => [7, 9, 2]
target_seq[3]:   => 0
input_seq[4] : e,  => [9, 2, 0]
target_seq[4]: n => 17
input_seq[5] : , n => [2, 0, 17]
target_seq[5]: o => 18
input_seq[6] :  no => [0, 17, 18]
target_seq[6]: r => 20
input_seq[7] : nor => [17, 18, 20]
target_seq[7]:   => 0
input_seq[8] : or  => [18, 20, 0]
target_seq[8]: o => 18
input_seq[9] : r o => [20, 0, 18]
target_seq[9]: t => 22
input_seq[10] :  ot => [0, 18, 22]
target_seq[10]:   => 0
input_seq[11] : ot  => [18, 22, 0]
target_seq[11]: t => 22
input_seq[12] : t t => [22, 0, 22]
target_seq[12]: o => 18
input_seq[13] :  to => [0, 22, 18]
target_seq[13]:   => 0
input_seq[14] : to  => [22, 18, 0]
target_seq[14]: b => 7
input_seq[15] : o b => [18, 0, 7]
target_seq[15]: e => 9
input_seq[16] :  be => [0, 7, 9]
target_seq[16]: , => 2
input_seq[17] : be, => [7, 9, 2]
targe

In [14]:
# 텐서로 변환
X_tensor = torch.LongTensor(X)
y_tensor = torch.LongTensor(y)


- [3] 모델 설계<hr>

In [15]:
### ===> LSTM 모델 정의
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # 마지막 시퀀스 출력만 사용
        return out
    

- [4]학습 준비<hr>

In [17]:
### ===> 하이퍼파라미터 설정
embedding_dim = 128
hidden_dim = 256
num_layers = 2
sequence_length = 10
num_epochs = 1000
learning_rate = 0.01

In [18]:
### ===> 모델 초기화
model = LSTMModel(vocab_size, embedding_dim, hidden_dim, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [20]:
### ===> 학습 진행 함수 
def train():
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        output = model(X_tensor)
        loss = criterion(output, y_tensor)
        loss.backward()
        optimizer.step()

        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

In [21]:

# 문장 생성 함수
def generate_text(model, start_str, gen_length=100):
    model.eval()
    generated = start_str
    input_seq = [char_to_idx[ch] for ch in start_str]
    input_tensor = torch.LongTensor(input_seq).unsqueeze(0)

    with torch.no_grad():
        for _ in range(gen_length):
            output = model(input_tensor)
            top_idx = output.argmax(dim=1)[0].item()  # 가장 높은 확률의 인덱스
            generated += idx_to_char[top_idx]
            input_seq.append(top_idx)
            input_tensor = torch.LongTensor(input_seq[-sequence_length:]).unsqueeze(0)

    return generated

- [5] 학습진행<hr>

In [22]:
train()

Epoch [100/1000], Loss: 0.0002
Epoch [200/1000], Loss: 0.0001
Epoch [300/1000], Loss: 0.0001
Epoch [400/1000], Loss: 0.0000
Epoch [500/1000], Loss: 0.0000
Epoch [600/1000], Loss: 0.0000
Epoch [700/1000], Loss: 0.0000
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.0000
Epoch [1000/1000], Loss: 0.0000


- [6] 테스트 <hr>

In [36]:
### ===> 문장 생성 예시
start_string = "To be"
generated_text = generate_text(model, start_string, gen_length=20)

print(generated_text)

To be, that is the questi
